## Download the data

A dataset could be created and downloaded using the new [views feature](https://docs.qcarchive.molssi.org/user_guide/datasets/caching.html).

Alternatively, download live from QCArchive (see [Retrieving results](https://docs.openforcefield.org/projects/qcsubmit/en/stable/examples/retrieving-results.html) for more).

In [1]:
from collections import defaultdict
from openff.units import unit
import numpy as np
import torch
import tqdm

In [2]:
from qcportal import PortalClient

qc_client = PortalClient("https://api.qcarchive.molssi.org:443", cache_dir=".")

In [3]:
from openff.qcsubmit.results import (
    BasicResultCollection,
    OptimizationResultCollection,
    TorsionDriveResultCollection,
)

In [10]:
# Pull down the torsion drive records from a dataset.
torsion_drive_result_collection = TorsionDriveResultCollection.from_server(
    client=qc_client,
    # small example dataset -- downloading and interacting with a dataset
    # can take a while!
    datasets="OpenFF Cresset Additional Coverage TorsionDrives v4.0",
    spec_name="default",
)

## Convert the data into a smee/descent-friendly format

The data needs to be postprocessed into a useful format for smee. Note, a lot of the functions here will benefit from parallelism as they can be very slow, some examples are provided in [Josh's repo](https://github.com/jthorton/SPICE-SMEE/blob/main/fit-v1).

In [11]:
import descent.targets.energy

bohr_to_angstrom = (1 * unit.bohr).m_as(unit.angstrom)
hartree_to_kcal = (1 * unit.hartree * unit.avogadro_constant).m_as(
    unit.kilocalories_per_mole
)
hartree_to_kcal

627.5094740630658

Ok we now want to create an [energy.Entry](https://simonboothroyd.github.io/descent/latest/reference/targets/energy/#descent.targets.energy.Entry) to be processed into a dataset. The final output is a [Huggingface dataset](https://huggingface.co/docs/datasets/en/index).

How to do this will differ for optimizations and torsiondrives slightly due to the structure of the code.
Here we look at torsiondrives, code for optimizations should be very similar but not need the `minimum_optimizations` part.

In [12]:
# create a dict to hold data by SMILES so conformers are mostly mapped together
# note: a more robust solution would use Molecule.are_isomorphic or similar method,
# but here we lazily just compare the smiles strings

data_by_smiles = defaultdict(list)
records_and_molecules = list(torsion_drive_result_collection.to_records())

In [ ]:
for td_record, molecule in tqdm.tqdm(records_and_molecules):
    # take only the optimized grid points
    for opt in td_record.minimum_optimizations.values():
        last = opt.trajectory[-1]
        last_mol = last.molecule
        mapped_smiles = last_mol.identifiers.canonical_isomeric_explicit_hydrogen_mapped_smiles
        coords = last_mol.geometry * bohr_to_angstrom
        energy = last.properties["return_energy"] * hartree_to_kcal
        gradient = np.array(last.properties["scf total gradient"]).reshape((-1, 3))
        forces = ((-gradient) * hartree_to_kcal / bohr_to_angstrom)
        entry = {
            "coords": coords,
            "energy": energy,
            "forces": forces,
        }
        data_by_smiles[mapped_smiles].append(entry)

  0%|                                                    | 0/23 [00:00<?, ?it/s]

In [8]:
# convert to smee's expected format
descent_entries = []
for mapped_smiles, entries in data_by_smiles.items():
    entry = {
        "smiles": mapped_smiles,
        "coords": torch.tensor([x["coords"] for x in entries]),
        "energy": torch.tensor([x["energy"] for x in entries]),
        "forces": torch.tensor([x["forces"] for x in entries]),
    }
    descent_entries.append(entry)

/var/folders/bf/2ytrzlmd6jbbgqs37bl9lld40000gn/T/ipykernel_55654/2096936775.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/miniforge3/conda-bld/libtorch_1739474920975/work/torch/csrc/utils/tensor_new.cpp:257.)
  "coords": torch.tensor([x["coords"] for x in entries]),


In [9]:
# this dataset can get downloaded, processed once, saved and reused
dataset = descent.targets.energy.create_dataset(entries=descent_entries)
dataset.save_to_disk("test-smee-data")

/Users/lily/micromamba/envs/smee-descent/lib/python3.11/site-packages/descent/targets/energy.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "coords": torch.tensor(entry["coords"]).flatten().tolist(),
/Users/lily/micromamba/envs/smee-descent/lib/python3.11/site-packages/descent/targets/energy.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "energy": torch.tensor(entry["energy"]).flatten().tolist(),
/Users/lily/micromamba/envs/smee-descent/lib/python3.11/site-packages/descent/targets/energy.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sou

## Assign parameters to molecules in the dataset

In [4]:
from openff.toolkit import Molecule, ForceField
import smee.converters

In [36]:
# uncomment if reloading data
# import datasets

# dataset = datasets.Dataset.load_from_disk("test-smee-data")

In [37]:
# reformat dataset lists to torch tensors
dataset.set_format('torch', columns=['energy', 'coords','forces'], output_all_columns=True)

In [38]:
# this is what a single entry looks like
dataset[0]

{'coords': tensor([ 1.4442e+00,  2.3826e+00,  1.2902e+00,  1.5731e+00,  1.1873e+00,
          8.9343e-01,  2.6362e+00,  5.0726e-01,  7.7696e-01,  2.3790e-01,
          5.2114e-01,  5.1677e-01,  2.0136e-03, -7.2446e-01,  6.2235e-02,
          1.0232e+00, -1.8039e+00, -1.9624e-01, -6.2302e-01,  1.1797e+00,
          6.4842e-01, -1.0379e+00, -9.9864e-01, -1.4715e-01,  8.1125e-01,
         -2.6950e+00,  4.1487e-01,  9.8896e-01, -2.1287e+00, -1.2477e+00,
          2.0187e+00, -1.4230e+00,  4.0675e-02,  1.4641e+00,  2.3277e+00,
          1.4532e+00,  1.6033e+00,  1.2213e+00,  8.5411e-01,  2.6749e+00,
          6.0657e-01,  5.7213e-01,  2.6757e-01,  5.8324e-01,  4.3581e-01,
          1.6428e-02, -6.7900e-01,  3.9613e-02,  1.0243e+00, -1.7875e+00,
         -1.3545e-01, -5.8370e-01,  1.2603e+00,  5.3138e-01, -1.0240e+00,
         -9.4722e-01, -1.7464e-01,  7.9145e-01, -2.6360e+00,  5.2631e-01,
          1.0007e+00, -2.1761e+00, -1.1654e+00,  2.0234e+00, -1.4061e+00,
          8.4873e-02,  1.474

Below we specify a starting force field.
Normally we would initialize parameters using the Modified Seminario method,
[example here](https://github.com/openforcefield/sage-2.2.1/blob/main/03_generate-initial-ff/create-msm-ff.py),
but here we just start from Sage 2.2.1.

[Josh's repo](https://github.com/jthorton/SPICE-SMEE/blob/main/fit-v1/training/001-expand_torsions.py)
has examples on expanding torsions too.

In [7]:
starting_ff = ForceField("openff-2.2.1.offxml")

In [9]:
all_smiles = []
interchanges = []
for entry in tqdm.tqdm(dataset):
    mol = Molecule.from_mapped_smiles(
        entry["smiles"],
        allow_undefined_stereo=True
    )
    all_smiles.append(entry["smiles"])
    interchange = starting_ff.create_interchange(mol.to_topology())
    interchanges.append(interchange)
    
smee_force_field, smee_topologies = smee.converters.convert_interchange(interchanges)
topologies = dict(zip(all_smiles, smee_topologies))

100%|███████████████████████████████████████████| 36/36 [01:17<00:00,  2.17s/it]


## Fit

Now we can set up and run the fit.

In [28]:
import descent.train
import descent.targets.energy

import math
import pathlib
import tensorboardX
import more_itertools


In [11]:
# specify which parameters to train
# and some details about them
# they're scaled so they're roughly on the same order of magnitude

parameters = {
    "Bonds": descent.train.ParameterConfig(
        cols=["k", "length"],
        scales={"k": 1e-2, "length": 1.0}, # normalize so roughly equal
        limits={"k":[0.0, None], "length": [0.0, None]}
        # the include/exclude types are Interchange PotentialKey.id's -- typically SMIRKS
        # include=[], <-- bonds to train. Not specifying trains all
        # exclude=[], <-- bonds NOT to train
    ),
    "Angles": descent.train.ParameterConfig(
        cols=["k", "angle"],
        scales={"k": 1e-2, "angle": 1.0},
        limits={"k": [0.0, None], "angle": [0.0, math.pi]}
    ),
    "ProperTorsions": descent.train.ParameterConfig(
        # fit ks
        cols=["k"],
        scales={"k": 1.0},
    ),
    "ImproperTorsions": descent.train.ParameterConfig(
        # fit ks
        cols=["k"],
        scales={"k": 1.0},
    )
}

In [12]:
trainable = descent.train.Trainable(
    force_field=smee_force_field,
    parameters=parameters,
    attributes={}
)

In [20]:
# optional below if you want cool tensorboard logging
def write_metrics(
        epoch: int,
        loss: torch.Tensor,
        loss_energy: torch.Tensor,
        loss_forces: torch.Tensor,
        writer: tensorboardX.SummaryWriter
):
    print(f"epoch={epoch} loss={loss.detach().item():.6f}", flush=True)

    writer.add_scalar("loss", loss.detach().item(), epoch)
    writer.add_scalar("loss_energy", loss_energy.detach().item(), epoch)
    writer.add_scalar("loss_forces", loss_forces.detach().item(), epoch)

    writer.add_scalar("rmse_energy", math.sqrt(loss_energy.detach().item()), epoch)
    writer.add_scalar("rmse_forces", math.sqrt(loss_forces.detach().item()), epoch)
    writer.flush()

Specify some hyperparameters, n_epochs is intentionally very low to guarantee fast execution.

In [22]:
N_EPOCHS = 10
LEARNING_RATE = 0.01
BATCH_SIZE = 500

In [23]:
# make directory to save files in
directory = pathlib.Path("my-smee-fit")
directory.mkdir(exist_ok=True, parents=True)


Run fit below.

In [1]:
# load tensorboard extension so we can view in notebook
%load_ext tensorboard

In [40]:
trainable_parameters = trainable.to_values()
device = trainable_parameters.device.type

with tensorboardX.SummaryWriter(str(directory)) as writer:
    optimizer = torch.optim.Adam([trainable_parameters], lr=LEARNING_RATE, amsgrad=True)
    dataset_indices = list(range(len(dataset)))

    for i in range(N_EPOCHS):
        ff = trainable.to_force_field(trainable_parameters)
        total_loss = torch.zeros(size=(1,), device=device)
        energy_loss = torch.zeros(size=(1,), device=device)
        force_loss = torch.zeros(size=(1,), device=device)
        grad = None
    
        for batch_ids in tqdm.tqdm(
            more_itertools.batched(dataset_indices, BATCH_SIZE),
            desc='Calculating energies',
            ncols=80, total=math.ceil(len(dataset) / BATCH_SIZE)
        ):
            batch = dataset.select(indices=batch_ids)
            true_batch_size = len(dataset)
            batch_configs = sum([len(d["energy"]) for d in batch])

            e_ref, e_pred, f_ref, f_pred = descent.targets.energy.predict(
                batch, ff, topologies, "mean"
            )   
            # L2 loss
            batch_loss_energy = ((e_pred - e_ref) ** 2).sum() / true_batch_size
            batch_loss_force = ((f_pred - f_ref) ** 2).sum() / true_batch_size

            # Equal sum of L2 loss on energies and forces
            batch_loss = batch_loss_energy + batch_loss_force

            (batch_grad, ) = torch.autograd.grad(batch_loss, trainable_parameters, create_graph=True)
            batch_grad = batch_grad.detach()
            if grad is None:
                grad = batch_grad
            else:
                grad += batch_grad
            
            # keep sum of squares to report MSE at the end
            total_loss += batch_loss.detach()
            energy_loss += batch_loss_energy.detach()
            force_loss += batch_loss_force.detach()
        
        trainable_parameters.grad = grad
        
        write_metrics(
            epoch=i, loss=total_loss, loss_energy=energy_loss,
            loss_forces=force_loss, writer=writer
        )

        optimizer.step()
        optimizer.zero_grad()

        if i % 10 == 0:
            torch.save(
                trainable.to_force_field(trainable_parameters),
                directory / f"force-field-epoch-{i}.pt"
            )

    torch.save(
        trainable.to_force_field(trainable_parameters),
        directory / "final-force-field.pt"
    )
    

Calculating energies: 100%|███████████████████████| 1/1 [00:00<00:00,  4.48it/s]

epoch=0 loss=269.888672



Calculating energies: 100%|███████████████████████| 1/1 [00:00<00:00,  6.57it/s]

epoch=1 loss=210.459198



Calculating energies: 100%|███████████████████████| 1/1 [00:00<00:00,  6.31it/s]

...

epoch=7 loss=150.993149



Calculating energies: 100%|███████████████████████| 1/1 [00:00<00:00,  6.79it/s]

epoch=8 loss=142.527145



Calculating energies: 100%|███████████████████████| 1/1 [00:00<00:00,  6.83it/s]

epoch=9 loss=137.984802


Metrics can be viewed in tensorboard below.

`tensorboard --logdir my-smee-fit` can also be run on command line instead of in the notebook.

In [2]:
%tensorboard --logdir my-smee-fit

Reusing TensorBoard on port 6006 (pid 58087), started 0:01:36 ago. (Use '!kill 58087' to kill it.)

## Convert back to OFFXML

In [52]:
for potential in smee_force_field.potentials:
    handler_name = potential.parameter_keys[0].associated_handler

    parameter_attrs = potential.parameter_cols
    parameter_units = potential.parameter_units

    if handler_name in ["Bonds", "Angles"]:
        handler = starting_ff.get_parameter_handler(handler_name)
        for i, opt_parameters in enumerate(potential.parameters):
            smirks = potential.parameter_keys[i].id
            ff_parameter = handler[smirks]
            opt_parameters = opt_parameters.detach().cpu().numpy()
            for j, (p, unit) in enumerate(zip(parameter_attrs, parameter_units)):
                setattr(ff_parameter, p, opt_parameters[j] * unit)

    elif handler_name in ["ProperTorsions"]:
        handler = starting_ff.get_parameter_handler(handler_name)
        k_index = parameter_attrs.index('k')
        p_index = parameter_attrs.index('periodicity')
        # we need to collect the k values into a list across the entries
        collection_data = defaultdict(dict)
        for i, opt_parameters in enumerate(potential.parameters):
            smirks = potential.parameter_keys[i].id
            ff_parameter = handler[smirks]
            opt_parameters = opt_parameters.detach().cpu().numpy()
            # find k and the periodicity
            k = opt_parameters[k_index] * parameter_units[k_index]
            p = int(opt_parameters[p_index])
            collection_data[smirks][p] = k
        # now update the force field
        for smirks, k_s in collection_data.items():
            ff_parameter = handler[smirks]
            k_mapped_to_p = [k_s[p] for p in ff_parameter.periodicity]
            ff_parameter.k = k_mapped_to_p

    elif handler_name in ["ImproperTorsions"]:
        k_index = parameter_attrs.index('k')
        handler = starting_ff.get_parameter_handler(handler_name)
        # we only fit the v2 terms for improper torsions so convert to list and set
        for i, opt_parameters in enumerate(potential.parameters):
            smirks = potential.parameter_keys[i].id
            ff_parameter = handler[smirks]
            opt_parameters = opt_parameters.detach().cpu().numpy()
            ff_parameter.k = [opt_parameters[k_index] * parameter_units[k_index]]

starting_ff.to_file("final-force-field.offxml")